In [21]:
import numpy as np
import pandas as pd
import nltk

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics.pairwise import cosine_similarity

# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', 500)
# pd.set_option('display.expand_frame_repr', False)


# STEAM GAME RECOMMENDER
**(Content Based Recommendation)**

In [2]:
df = pd.read_csv("./steam_games.csv")
df.head(2)

,url,types,name,desc_snippet,recent_reviews,all_reviews,release_date,developer,publisher,popular_tags,game_details,languages,achievements,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price,discount_price
0,https://store.steampowered.com/app/379720/DOOM/,app,DOOM,Now includes all three premium DLC packs (Unto...,"Very Positive,(554),- 89% of the 554 user revi...","Very Positive,(42,550),- 92% of the 42,550 use...","May 12, 2016",id Software,"Bethesda Softworks,Bethesda Softworks","FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","Single-player,Multi-player,Co-op,Steam Achieve...","English,French,Italian,German,Spanish - Spain,...",54.0,Action,"About This Game Developed by id software, the...",NaN,"Minimum:,OS:,Windows 7/8.1/10 (64-bit versions...","Recommended:,OS:,Windows 7/8.1/10 (64-bit vers...",$19.99,$14.99
1,https://store.steampowered.com/app/578080/PLAY...,app,PLAYERUNKNOWN'S BATTLEGROUNDS,PLAYERUNKNOWN'S BATTLEGROUNDS is a battle roya...,"Mixed,(6,214),- 49% of the 6,214 user reviews ...","Mixed,(836,608),- 49% of the 836,608 user revi...","Dec 21, 2017",PUBG Corporation,"PUBG Corporation,PUBG Corporation","Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats","English,Korean,Simplified Chinese,French,Germa...",37.0,"Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,Mature Content Description The developers de...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40833 entries, 0 to 40832
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   url                       40833 non-null  object 
 1   types                     40831 non-null  object 
 2   name                      40817 non-null  object 
 3   desc_snippet              27612 non-null  object 
 4   recent_reviews            2706 non-null   object 
 5   all_reviews               28470 non-null  object 
 6   release_date              37654 non-null  object 
 7   developer                 40490 non-null  object 
 8   publisher                 35733 non-null  object 
 9   popular_tags              37888 non-null  object 
 10  game_details              40313 non-null  object 
 11  languages                 40797 non-null  object 
 12  achievements              12194 non-null  float64
 13  genre                     40395 non-null  object 
 14  game_d

In [4]:
df1 = df[["name", "desc_snippet", "game_description", "popular_tags","genre"]]
df2 = pd.DataFrame(df1.fillna(""))

df2.head(5)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40833 entries, 0 to 40832
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   name              40833 non-null  object
 1   desc_snippet      40833 non-null  object
 2   game_description  40833 non-null  object
 3   popular_tags      40833 non-null  object
 4   genre             40833 non-null  object
dtypes: object(5)
memory usage: 1.6+ MB


In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vikto\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
stopwords = nltk.corpus.stopwords.words('english')
stopwords
# stop_extensions = ['']
# stopwords.extend(stop_extensions)

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [7]:
tfidf = TfidfVectorizer(
                     use_idf=True,
                     min_df=0.01,
                     ngram_range=(1,3),
                     stop_words=stopwords
                    )

# объединяем столбики с описаниями, чтобы получить больше информации о каждой игре и обработать тфидф-ом
tfidf_matrix = tfidf.fit_transform(df2["desc_snippet"] + "\n" + df2["game_description"])

In [23]:
count_vect = CountVectorizer(stop_words=stopwords)

dataset = count_vect.fit_transform(df2["desc_snippet"] + "\n" + df2["game_description"])

In [25]:
lda = LatentDirichletAllocation(n_components = 60,
                 max_iter=30,
                 n_jobs=6,
                 learning_method='batch',
                 verbose=1)
lda.fit(dataset)

# lda = LatentDirichletAllocation(n_components=5, random_state=0)

# lda_matrix = lda.fit_transform(df2["desc_snippet"] + "\n" + df2["game_description"])

iteration: 1 of max_iter: 30
iteration: 2 of max_iter: 30
iteration: 3 of max_iter: 30
iteration: 4 of max_iter: 30
iteration: 5 of max_iter: 30
iteration: 6 of max_iter: 30
iteration: 7 of max_iter: 30
iteration: 8 of max_iter: 30
iteration: 9 of max_iter: 30
iteration: 10 of max_iter: 30
iteration: 11 of max_iter: 30
iteration: 12 of max_iter: 30
iteration: 13 of max_iter: 30
iteration: 14 of max_iter: 30
iteration: 15 of max_iter: 30
iteration: 16 of max_iter: 30
iteration: 17 of max_iter: 30
iteration: 18 of max_iter: 30
iteration: 19 of max_iter: 30
iteration: 20 of max_iter: 30
iteration: 21 of max_iter: 30
iteration: 22 of max_iter: 30
iteration: 23 of max_iter: 30
iteration: 24 of max_iter: 30
iteration: 25 of max_iter: 30
iteration: 26 of max_iter: 30
iteration: 27 of max_iter: 30
iteration: 28 of max_iter: 30
iteration: 29 of max_iter: 30
iteration: 30 of max_iter: 30


LatentDirichletAllocation(max_iter=30, n_components=60, n_jobs=6, verbose=1)

In [35]:
hm = lda.transform(dataset)
hm.shape

(40833, 60)

In [36]:
cosine_sim = cosine_similarity(hm,
        hm)

In [37]:
cosine_sim.shape

(40833, 40833)

In [31]:
cosine_sim = cosine_similarity(tfidf_matrix,
                               tfidf_matrix)

cosine_sim.shape

(40833, 40833)

In [10]:
def steam_game_recommender(title, cosine_sim, dataframe):
    indices = pd.Series(dataframe.index, index=dataframe['name'])
    indices = indices[~indices.index.duplicated(keep='last')]
    game_index = indices[title]
    similarity_scores = pd.DataFrame(cosine_sim[game_index], columns=["score"])
    game_indices = similarity_scores.sort_values("score", ascending=False)[1:11].index
    return dataframe['name'].iloc[game_indices]

In [11]:
steam_game_recommender("Frostpunk", cosine_sim, df2)

28071                       Urban Empire
40029                       Metropolisim
20650                  Road to your City
29981    City Of Jade: Imperial Frontier
3879              Holyday City: Reloaded
36582                        Living Dark
3495            Constructor Classic 1997
6915                   25 Cadre of Death
27116                     City Life 2008
808                        The Long Dark
Name: name, dtype: object

In [12]:
steam_game_recommender("EVE Online", cosine_sim, df2)

39323                             Icarus Online
639                                     Elsword
4065                               CABAL Online
37406                             Star Sonata 2
24979                                  Millidor
40463                           Legends of Aria
27049    Shroud of the Avatar: Forsaken Virtues
26309                                  StarMade
40739                           CSC | Space MMO
336                                  Brawlhalla
Name: name, dtype: object

In [13]:
steam_game_recommender("DayZ", cosine_sim, df2)

24357                          Hurricane
9335              ICY: Frostbite Edition
39787                             Nostos
433                           Miscreated
3901     Dinosaurs Prehistoric Survivors
2420              Infestation: The New Z
40296               Survive the Blackout
517                       XERA: Survival
178                             The Isle
2092                           The Crew™
Name: name, dtype: object